# RQ1 - Eyetracking

## Import Libraries

In [1]:
import sys
sys.path.append('../data/StudyPeitek/dataEvaluation/')

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import utils.GenSnippetsLib as gsl
import math
import json


## Import Fixation Data

In [2]:
# function to convert a string seperated by whitespace characters back to python list
def string_to_list_string(data):
    data = data.replace(' ', ',')
    data = data.replace('\n', ',')
    data = ','.join([element for element in data.split(",") if len(element) > 0])
    if data[1] == ",":
        data = "[" + data[2:]
    return data

# read in the fixation data
df_fixation = pd.read_csv('results/ogama_fixations_peitek.csv', sep=";")
# kick out outliers
df_fixation = df_fixation[df_fixation["IsOutlier"] == False]
df_fixation = df_fixation.drop(columns=["IsOutlier", "Behavioral"])

# transform fixation strings to lists
df_fixation["Fixation_startT"] = df_fixation["Fixation_startT"].apply(string_to_list_string)
df_fixation["Fixation_endT"] = df_fixation["Fixation_endT"].apply(string_to_list_string)
df_fixation["Fixation_x"] = df_fixation["Fixation_x"].apply(string_to_list_string)
df_fixation["Fixation_y"] = df_fixation["Fixation_y"].apply(string_to_list_string)
df_fixation["Fixation_x_range"] = df_fixation["Fixation_x_range"].apply(string_to_list_string)
df_fixation["Fixation_y_range"] = df_fixation["Fixation_y_range"].apply(string_to_list_string)

# Token Based Metrics

### Read in the Generator for Token Based Metrics to get The BoundingBoxes and Indices of each Token per Algorithm

In [3]:
# Print the files in ../data/24/CodeSnippets/fonts/ttf/ for the available fonts
import os
print(os.listdir('../data/24/CodeSnippets/fonts/ttf/'))
# Get Token based AOIs
snippets = df_fixation["Algorithm"].unique()
df_token_aois = pd.DataFrame(columns=["Algorithm", "Token", "TokenIdx", "BoundingBox"])
for snippet in tqdm(snippets):
    aoi_token_generator = f"../data/24/CodeSnippets/Generators_Labeled/Generators/{snippet}_ast.json"


    image, aoi_list = gsl.create_image(aoi_token_generator, font_path="/../data/24/CodeSnippets/fonts/ttf/")
    height, width = image.size
    width_offset = int(1920 * 0.5) - int(height / 2)
    height_offset = int(1080 * 0.5) - int(width / 2)
    aoi_clustered = []
    current_left = None
    current_top = None
    current_right = None
    current_bottom = None
    current_aoi = None
    color = None
    for letter in aoi_list:
        if len(letter["AOI"]) == 1 or letter["letter"] == '\n':
            if current_aoi is not None:
                aoi_clustered.append(
                    (len(aoi_clustered), current_aoi, current_left, current_top, current_right, current_bottom, color))
            current_aoi = None
            color = None
            current_left = None
            current_top = None
            current_right = None
            current_bottom = None
            continue
        if current_aoi is None:
            current_aoi = letter["AOI"][1]
            color = letter["color"]
            current_left = letter["BoundingBox"][0]
            current_top = letter["BoundingBox"][1]
            current_right = letter["BoundingBox"][2]
            current_bottom = letter["BoundingBox"][3]
        elif current_aoi == letter["AOI"][1]:
            current_left = min(current_left, letter["BoundingBox"][0])
            current_top = min(current_top, letter["BoundingBox"][1])
            current_right = max(current_right, letter["BoundingBox"][2])
            current_bottom = max(current_bottom, letter["BoundingBox"][3])
        else:
            aoi_clustered.append(
                (len(aoi_clustered), current_aoi, current_left, current_top, current_right, current_bottom, color))
            current_aoi = letter["AOI"][1]
            color = letter["color"]
            current_left = letter["BoundingBox"][0]
            current_top = letter["BoundingBox"][1]
            current_right = letter["BoundingBox"][2]
            current_bottom = letter["BoundingBox"][3]

    for token in aoi_clustered:
        df_token_aois.loc[len(df_token_aois)] = [snippet, token[1], token[0],
                                                 (token[2] + width_offset,
                                                  token[3] + height_offset,
                                                  token[4] + width_offset,
                                                  token[5] + height_offset)]
df_token_aois

['JetBrainsMono-Bold.ttf', 'JetBrainsMono-BoldItalic.ttf', 'JetBrainsMono-ExtraBold.ttf', 'JetBrainsMono-ExtraBoldItalic.ttf', 'JetBrainsMono-ExtraLight.ttf', 'JetBrainsMono-ExtraLightItalic.ttf', 'JetBrainsMono-Italic.ttf', 'JetBrainsMono-Light.ttf', 'JetBrainsMono-LightItalic.ttf', 'JetBrainsMono-Medium.ttf', 'JetBrainsMono-MediumItalic.ttf', 'JetBrainsMono-Regular.ttf', 'JetBrainsMono-Thin.ttf', 'JetBrainsMono-ThinItalic.ttf', 'JetBrainsMonoNL-Bold.ttf', 'JetBrainsMonoNL-BoldItalic.ttf', 'JetBrainsMonoNL-ExtraBold.ttf', 'JetBrainsMonoNL-ExtraBoldItalic.ttf', 'JetBrainsMonoNL-ExtraLight.ttf', 'JetBrainsMonoNL-ExtraLightItalic.ttf', 'JetBrainsMonoNL-Italic.ttf', 'JetBrainsMonoNL-Light.ttf', 'JetBrainsMonoNL-LightItalic.ttf', 'JetBrainsMonoNL-Medium.ttf', 'JetBrainsMonoNL-MediumItalic.ttf', 'JetBrainsMonoNL-Regular.ttf', 'JetBrainsMonoNL-Thin.ttf', 'JetBrainsMonoNL-ThinItalic.ttf']


  0%|          | 0/32 [00:00<?, ?it/s]

AOI not closed - None
AOI not closed - None
AOI not closed - None


,Algorithm,Token,TokenIdx,BoundingBox
0,IsPrime,Modifier,0,"(798, 468, 846, 482)"
1,IsPrime,Modifier,1,"(854, 468, 902, 479)"
2,IsPrime,BasicType,2,"(910, 469, 966, 479)"
3,IsPrime,Identifier,3,"(974, 468, 1030, 479)"
4,IsPrime,Separator,4,"(1030, 467, 1038, 481)"
...,...,...,...,...
2488,Rectangle,Keyword,82,"(977, 677, 1009, 688)"
2489,Rectangle,Separator,83,"(1009, 685, 1017, 688)"
2490,Rectangle,Identifier,84,"(1017, 677, 1065, 691)"
2491,Rectangle,Separator,85,"(1065, 676, 1089, 691)"


### Check which Fixation of which Participant is in which Token

In [4]:
df_token_fixation_per_participant = pd.DataFrame([], columns=["Algorithm", "Participant", "FixationNumber",
                                                            "FixationDuration", "TokenIdx"])
participants = df_fixation["Participant"].unique()
for snippet in tqdm(snippets):
    df_token_per_algo = df_token_aois[df_token_aois["Algorithm"] == snippet]

    for participant in participants:
        df_fixation_participant = df_fixation[
            (df_fixation["Algorithm"] == snippet) & (df_fixation["Participant"] == participant)]
        if len(df_fixation_participant) == 0:
            continue
        start_times = eval(df_fixation_participant["Fixation_startT"].values[0])
        end_times = eval(df_fixation_participant["Fixation_endT"].values[0])
        x_coordinates = eval(df_fixation_participant["Fixation_x"].values[0])
        y_coordinates = eval(df_fixation_participant["Fixation_y"].values[0])
        x_range = eval(df_fixation_participant["Fixation_x_range"].values[0])
        y_range = eval(df_fixation_participant["Fixation_y_range"].values[0])
        idx_values = range(len(start_times))

        for (fix_idx, start, end, x, y, x_range, y_range) in zip(idx_values, start_times, end_times, x_coordinates, y_coordinates, x_range, y_range):
            low_x = int(float(x) - math.ceil(float(x_range)))
            low_y = int(float(y) - math.ceil(float(y_range)))
            high_x = int(float(x) + math.ceil(float(x_range)))
            high_y = int(float(y) + math.ceil(float(y_range)))
            possible_coordinates = [(x, y) for x in range(low_x, high_x + 1) for y in range(low_y, high_y + 1)]

            found = False
            for idx, row in df_token_per_algo.iterrows():
                token_idx = row["TokenIdx"]
                bounding_box = row["BoundingBox"]

                for possible_x, possible_y in possible_coordinates:
                    if bounding_box[0] <= possible_x <= bounding_box[2] and bounding_box[1] <= possible_y <=bounding_box[3]:
                        df_token_fixation_per_participant.loc[len(df_token_fixation_per_participant)] = [snippet, participant,fix_idx,end - start,token_idx]
                        found = True
                        break
                if found:
                    break

df_token_fixation_per_participant

  0%|          | 0/32 [00:00<?, ?it/s]

,Algorithm,Participant,FixationNumber,FixationDuration,TokenIdx
0,IsPrime,1,4,172,0
1,IsPrime,1,6,308,1
2,IsPrime,1,12,240,10
3,IsPrime,1,18,180,16
4,IsPrime,1,24,296,9
...,...,...,...,...,...
21421,Rectangle,68,85,304,73
21422,Rectangle,70,12,168,33
21423,Rectangle,70,17,176,36
21424,Rectangle,70,20,196,22


### Transform the Data to a Fixation/ Refixation split by Participant

In [5]:
df_token = df_token_aois.copy()
df_token = df_token.drop(["BoundingBox", "Token"], axis=1)
for participant in participants:
    df_token.loc[:, f"TokenFixation_P{participant}"] = [[] for _ in range(len(df_token))]
    df_token.loc[:, f"TokenReFixation_P{participant}"] = [[] for _ in range(len(df_token))]



prev_participant = df_token_fixation_per_participant["Participant"].iloc[0]
prev_token_idx = df_token_fixation_per_participant["TokenIdx"].iloc[0]
prev_algorithm = df_token_fixation_per_participant["Algorithm"].iloc[0]
fixations = []
re_fixation = False
for idx, row in tqdm(df_token_fixation_per_participant.iterrows(), total=len(df_token_fixation_per_participant)):
    participant = row["Participant"]
    token_idx = row["TokenIdx"]
    algorithm = row["Algorithm"]
    FixationDuration = row["FixationDuration"]

    # fixation switches
    if prev_participant != participant or prev_token_idx != token_idx:
        index = df_token[(df_token["TokenIdx"] == prev_token_idx) & (df_token["Algorithm"] == prev_algorithm)].index[0]
        if re_fixation:
            re_fixations = df_token.loc[index, f"TokenReFixation_P{prev_participant}"]
            re_fixations.extend(fixations.copy())
            df_token.at[index, f"TokenReFixation_P{prev_participant}"] = re_fixations.copy()
        else:
            df_token.at[index, f"TokenFixation_P{prev_participant}"] = fixations.copy()
        fixations = []
        # possible new fixation
        re_fixation = False

    sub_frame = df_token[(df_token["TokenIdx"] == token_idx) & (df_token["Algorithm"] == algorithm)]
    if len(sub_frame) == 0:
        raise Exception(f"No Token found for {token_idx} in Algorithm {algorithm}")
    len_of_fixation = len(sub_frame[f"TokenFixation_P{participant}"].iloc[0])
    if re_fixation == False and len_of_fixation > 0 and len(fixations) == 0:
        re_fixation = True

    fixations.append(FixationDuration)
    prev_participant = participant
    prev_token_idx = token_idx
    prev_algorithm = algorithm

df_token

  0%|          | 0/21426 [00:00<?, ?it/s]

,Algorithm,TokenIdx,TokenFixation_P1,TokenReFixation_P1,TokenFixation_P2,TokenReFixation_P2,TokenFixation_P3,TokenReFixation_P3,TokenFixation_P4,TokenReFixation_P4,...,TokenFixation_P66,TokenReFixation_P66,TokenFixation_P67,TokenReFixation_P67,TokenFixation_P68,TokenReFixation_P68,TokenFixation_P70,TokenReFixation_P70,TokenFixation_P71,TokenReFixation_P71
0,IsPrime,0,[172],[],"[204, 92]",[168],[],[],[],[],...,[],[],[128],[],[],[],[],[],[],[]
1,IsPrime,1,[308],[],"[204, 188]",[368],[],[],[],[],...,[],[],[],[],[320],[],[],[],[],[]
2,IsPrime,2,[],[],[196],"[204, 180, 244, 240]",[],[],"[276, 176]",[],...,[],[],[684],[],[668],"[204, 324, 396]",[],[],[],[]
3,IsPrime,3,[212],[],[532],[],[],[],[],[],...,[96],[424],"[700, 200]",[],[120],[],[],[],[],[]
4,IsPrime,4,[],[],[],[],[],[],[],[],...,[],[],[],[],[100],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2488,Rectangle,82,[316],[380],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
2489,Rectangle,83,[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
2490,Rectangle,84,[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
2491,Rectangle,85,[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]


### Calculate the Token Based Eyetracking Metrics

In [30]:
# Melt the Dataframe to be thinner so that we have Algorithm TokenIdx and Participant and Keys per Fixation / Refixation
df_token_melted = pd.melt(df_token, id_vars=["Algorithm", "TokenIdx"], var_name="KindOfFixation", value_name="FixationDurations")

# Classify Participant and the Kind and Number of Fixations / Refixations
df_token_melted["Participant"] = df_token_melted["KindOfFixation"].apply(lambda x: int(x.split("_")[1][1:]))
df_token_melted["KindOfFixation"] = df_token_melted["KindOfFixation"].apply(lambda x: x.split("_")[0])
df_token_melted["KindOfFixation"] = df_token_melted["KindOfFixation"].apply(lambda x: "Fixation" if x == "TokenFixation" else "ReFixation")
df_token_melted["NumberOfFixations"] = df_token_melted["FixationDurations"].apply(lambda x: len(x))

# Get the number of Participants for further calculations
number_of_participants = len(participants)

# Calculate the First Fixation Duration per Participant per Token
df_token_melted["FirstFixationDuration"] = None
df_token_melted.loc[df_token_melted["KindOfFixation"] == "Fixation" , "FirstFixationDuration"] = df_token_melted["FixationDurations"]\
    .apply(lambda x: x[0] if len(x) > 0 else None)

# Calculate the Single Fixation Duration per Participant per Token
df_token_melted["SingleFixationDuration"] = None
df_token_melted.loc[df_token_melted["KindOfFixation"] == "Fixation" , "SingleFixationDuration"] = df_token_melted["FixationDurations"]\
    .apply(lambda x: x[0] if len(x) == 1 else None)

# Calculate the Gaze Duration per Participant per Token
df_token_melted["GazeDuration"] = None
df_token_melted.loc[df_token_melted["KindOfFixation"] == "Fixation" , "GazeDuration"] = df_token_melted["FixationDurations"]\
    .apply(lambda x: sum(x) if len(x) > 0 else None)

# Calculate the Total Time per Participant per Token
df_token_melted_total_time = df_token_melted.groupby(["Participant", "Algorithm", "TokenIdx"])\
    .agg({"FixationDurations": lambda x: sum(x.values.sum())})
df_token_melted_total_time = df_token_melted_total_time.rename(columns={"FixationDurations": "TotalTime"})

# Merge the Dataframes
df_token_melted = pd.merge(df_token_melted, df_token_melted_total_time, on=["Participant", "Algorithm", "TokenIdx"], how="left")

# Cast the Dataframes to the right datatype
df_token_melted["FirstFixationDuration"] = df_token_melted["FirstFixationDuration"].astype(float)
df_token_melted["SingleFixationDuration"] = df_token_melted["SingleFixationDuration"].astype(float)
df_token_melted["GazeDuration"] = df_token_melted["GazeDuration"].astype(float)
df_token_melted["TotalTime"] = df_token_melted["TotalTime"].astype(float).replace(0, np.nan)

# Read in the Skilllevel
df_skill = pd.read_csv(f"../data/24/dataEvaluation/data/filteredData/filtered_data.csv")
df_skill = df_skill[["Participant", "SkillScore"]]
df_skill = df_skill.drop_duplicates()

# Merge the Dataframes to combine metrics with the Skilllevel
df_metrics_skill = pd.merge(df_token_melted, df_skill, on=["Participant"], how="left")

# Helper Methods for the Metrics
def get_no_fixations(df):
    df_fixations = df[df["KindOfFixation"] == "Fixation"]
    # remove every entry from df fixations on ["Algorithm", "TokenIdx"] where there is a refixation6
    df_fixations = df_fixations[df_fixations["NumberOfFixations"] == 0]
    return df_fixations


def get_single_fixations(df):
    df_fixations = df[df["KindOfFixation"] == "Fixation"]
    df_refixations = df[df["KindOfFixation"] == "ReFixation"]
    df_refixations = df_refixations[df_refixations["NumberOfFixations"] > 0]
    # remove every entry from df fixations on ["Algorithm", "TokenIdx"] where there is a refixation6
    df_fixations = df_fixations[~df_fixations["TokenIdx"].isin(df_refixations["TokenIdx"].values)]
    # remove every entry from df fixations on where Number Of Fixations is not 1
    df_fixations = df_fixations[df_fixations["NumberOfFixations"] == 1]
    return df_fixations

def get_multiple_fixations(df):
    df_fixations = df[df["KindOfFixation"] == "Fixation"]
    df_refixations = df[df["KindOfFixation"] == "ReFixation"]
    df_refixations = df_refixations[df_refixations["NumberOfFixations"] > 0]
    # remove every entry from df fixations on ["Algorithm", "TokenIdx"] where there is a refixation6
    df_fixations = df_fixations[(df_fixations["TokenIdx"].isin(df_refixations["TokenIdx"].values)) & (df_fixations["NumberOfFixations"] >= 1)]
    return df_fixations

def get_fixations(df):
    df_fixations = df[df["KindOfFixation"] == "Fixation"]
    # remove every entry from df fixations on where no Fixation is found
    df_fixations = df_fixations[df_fixations["NumberOfFixations"] >= 1]
    return df_fixations



# dataframe for number of fixations per participant
number_of_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])["NumberOfFixations"].count()
number_of_fixation_per_algorithm = number_of_fixation_per_algorithm.reset_index()

# dataframe for number of tokens per algorithm
number_of_tokens_per_algorithm = df_metrics_skill.groupby(["Algorithm"])["TokenIdx"].max()
number_of_tokens_per_algorithm = number_of_tokens_per_algorithm.reset_index()

# dataframe for number of tokens with no fixation per algorithm per participant
df_no_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])\
    .apply(get_no_fixations)\
    .drop(["Algorithm"], axis=1)

# dataframe for number of tokens with only one fixation per algorithm per participant
df_single_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])\
    .apply(get_single_fixations)\
    .drop(["Algorithm"], axis=1)

# dataframe for number of tokens with more than one fixation per algorithm per participant
df_multiple_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])\
    .apply(get_multiple_fixations)\
    .drop(["Algorithm"], axis=1)

# dataframe for number of tokens with more or equal than one fixation per algorithm per participant
df_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])\
    .apply(get_fixations)\
    .drop(["Algorithm"], axis=1)

# Reformat the dataframes
no_fixation_per_algorithm = df_no_fixation_per_algorithm[["TokenIdx"]]
no_fixation_per_algorithm = no_fixation_per_algorithm.reset_index().drop(["level_2"], axis=1)

single_fixation_per_algorithm = df_single_fixation_per_algorithm[["TokenIdx"]]
single_fixation_per_algorithm = single_fixation_per_algorithm.reset_index().drop(["level_2"], axis=1)

multiple_fixation_per_algorithm = df_multiple_fixation_per_algorithm[["TokenIdx"]]
print(df_multiple_fixation_per_algorithm)
multiple_fixation_per_algorithm = multiple_fixation_per_algorithm.reset_index().drop(["level_2"], axis=1)
print(multiple_fixation_per_algorithm['Participant'].unique())

fixations_per_algorithm = df_fixation_per_algorithm[["TokenIdx"]]
fixations_per_algorithm = fixations_per_algorithm.reset_index().drop(["level_2"], axis=1)

# Helper Method for Probability Metrics
def group_len_divided_by_number(current_df, counting_df):
    algorithm = current_df["Algorithm"].iloc[0]
    number_of_tokens = counting_df[counting_df["Algorithm"] == algorithm]["TokenIdx"].iloc[0]
    value = len(current_df) / number_of_tokens
    return len(current_df) / (number_of_tokens + 1)

# Calculate the Metrics per Participant
# Probability of no fixation
df_no_fixation_probability = no_fixation_per_algorithm.groupby(["Participant", "Algorithm"]).apply(lambda df: group_len_divided_by_number(df, number_of_tokens_per_algorithm))
df_no_fixation_probability = df_no_fixation_probability.reset_index()

# Probability of single fixation
df_single_fixation_probability = single_fixation_per_algorithm.groupby(["Participant", "Algorithm"]).apply(lambda df: group_len_divided_by_number(df, number_of_tokens_per_algorithm))
df_single_fixation_probability = df_single_fixation_probability.reset_index()

# Probability of multiple fixation
df_multiple_fixation_probability = multiple_fixation_per_algorithm.groupby(["Participant", "Algorithm"]).apply(lambda df: group_len_divided_by_number(df, number_of_tokens_per_algorithm))
df_multiple_fixation_probability = df_multiple_fixation_probability.reset_index()


#Print the difference between the number of participants and the number of participants in the df_multiple_fixation_probability
print(df_multiple_fixation_probability['Participant'].unique())

# Probability of fixation
df_fixation_probability = fixations_per_algorithm.groupby(["Participant", "Algorithm"]).apply(lambda df: group_len_divided_by_number(df, number_of_tokens_per_algorithm))
df_fixation_probability = df_fixation_probability.reset_index()

# Calculate the means for the metrics per algorithm
df_no_fixation_probability = df_no_fixation_probability.groupby(["Participant"]).mean(numeric_only=True)
df_single_fixation_probability = df_single_fixation_probability.groupby(["Participant"]).mean(numeric_only=True)
df_multiple_fixation_probability = df_multiple_fixation_probability.groupby(["Participant"]).mean(numeric_only=True)
# Add the value 0 for participant 41 as he has no data that can be used



if 41 not in df_multiple_fixation_probability.index:
    df_multiple_fixation_probability.loc[41] = 0
# Sort the DataFrame by Participant
df_multiple_fixation_probability = df_multiple_fixation_probability.sort_index()
print(df_multiple_fixation_probability)
#df_multiple_fixation_probability.at[41, "Participant"] = 0
df_fixation_probability = df_fixation_probability.groupby(["Participant"]).mean(numeric_only=True)


# Raw Durations Metrics
# Duration of first fixation
df_first_fixation = df_metrics_skill[~df_metrics_skill["FirstFixationDuration"].isnull()]
df_first_fixation = df_first_fixation.groupby(["Participant"])["FirstFixationDuration"].mean()

# Duration of single fixation
df_single_fixation = df_metrics_skill[~df_metrics_skill["SingleFixationDuration"].isnull()]
df_single_fixation = df_single_fixation.groupby(["Participant"])["SingleFixationDuration"].mean()

# Duration of gaze duration
df_gaze_duration = df_metrics_skill[~df_metrics_skill["GazeDuration"].isnull()]
df_gaze_duration = df_gaze_duration.groupby(["Participant"])["GazeDuration"].mean()

# Total Time
df_total_time = df_metrics_skill[~df_metrics_skill["TotalTime"].isnull()]
df_total_time = df_total_time.groupby(["Participant"])["TotalTime"].mean()

# Put every metric dataframe together into one
df_combined = pd.DataFrame({"FirstFixationDuration": df_first_fixation.values,
                            "SingleFixationDuration": df_single_fixation.values,
                            "GazeDuration": df_gaze_duration.values,
                            "TotalTime": df_total_time.values,
                            "TokenNoFixationProbability": df_no_fixation_probability.values.reshape(37, ),
                            "TokenSingleFixationProbability": df_single_fixation_probability.values.reshape(37, ),
                            "TokenMultipleFixationProbability": df_multiple_fixation_probability.values.reshape(37, ),
                            "TokenFixationProbability": df_fixation_probability.values.reshape(37, ),
                            "Skill": df_metrics_skill.groupby(["Participant"])["SkillScore"].mean().values})
# get spearman correlation for metrics and skill level
df_combined.corrwith(df_combined["Skill"])

                                 TokenIdx KindOfFixation   
Participant Algorithm                                      
1           Ackerman     2271           0       Fixation  \
                         2284          13       Fixation   
            ArrayAverage 1431           0       Fixation   
            BinToDecimal 457            0       Fixation   
                         459            2       Fixation   
...                                   ...            ...   
71          SmallGauss   181569         7       Fixation   
                         181573        11       Fixation   
            Vehicle      180588         1       Fixation   
                         180589         2       Fixation   
                         180590         3       Fixation   

                                    FixationDurations  Participant   
Participant Algorithm                                                
1           Ackerman     2271              [392, 212]            1  \
         

FirstFixationDuration              -0.000291
SingleFixationDuration              0.005996
GazeDuration                       -0.076264
TotalTime                          -0.291047
TokenNoFixationProbability          0.114338
TokenSingleFixationProbability     -0.286528
TokenMultipleFixationProbability   -0.390208
TokenFixationProbability           -0.356666
Skill                               1.000000
dtype: float64

In [31]:
df_combined

,FirstFixationDuration,SingleFixationDuration,GazeDuration,TotalTime,TokenNoFixationProbability,TokenSingleFixationProbability,TokenMultipleFixationProbability,TokenFixationProbability,Skill
0,271.142180,268.613793,308.156398,581.744076,0.707979,0.159176,0.126856,0.292021,0.331385
1,256.270567,254.659044,306.230347,502.846435,0.756208,0.154986,0.110228,0.278620,0.379187
2,314.602339,315.709877,334.438596,537.461988,0.905097,0.145165,0.076175,0.202461,0.311264
3,269.721248,269.062370,291.565302,485.185185,0.769454,0.130669,0.103375,0.230546,0.424727
4,309.167123,302.721212,349.095890,500.424658,0.816478,0.117943,0.086634,0.202507,0.313031
5,258.831429,262.720859,277.597143,497.957143,0.848820,0.155645,0.127422,0.284574,0.315932
6,161.395349,161.951220,168.558140,392.000000,0.981711,0.029346,0.060057,0.041803,0.420873
7,277.563187,275.717647,304.662088,426.456044,0.830102,0.108726,0.074787,0.181225,0.350392
8,225.050000,226.648746,245.156667,501.676667,0.827637,0.140283,0.147912,0.290296,0.178206
9,269.234834,270.031646,295.888454,625.322896,0.790467,0.129804,0.124532,0.248335,0.309233


In [32]:
# Save the Dataframe to a CSV
df_combined.to_csv("results/ogama_metrics.csv", index=False)

# AOI based analysis

### Read in the Generator for AOI Based Metrics to get The BoundingBoxes and Indices of each Token per Algorithm

In [8]:
# Get Token based AOIs
snippets = df_fixation["Algorithm"].unique()
df_aois = pd.DataFrame(columns=["Algorithm", "AOI", "AOIIdx", "BoundingBox"])
for snippet in tqdm(snippets):
    aoi_token_generator = f"../data/24/CodeSnippets/Generators_Labeled/Generators/{snippet}.json"
    try:
        image, aoi_list = gsl.create_image(aoi_token_generator, font_path="/../data/24/CodeSnippets/fonts/ttf/")
    except:
        print(f"{snippet} failed")
        image, aoi_list = gsl.create_image(aoi_token_generator, font_path="/../data/24/CodeSnippets/fonts/ttf/", logging=True)
        break
    height, width = image.size
    width_offset = int(1920 * 0.5) - int(height / 2)
    height_offset = int(1080 * 0.5) - int(width / 2)
    aoi_clustered = []
    current_left = []
    current_top = []
    current_right = []
    current_bottom = []
    current_aoi = []
    color = []
    for letter in aoi_list:
        # Close AOI
        if letter['letter'] in " \t\n":
            continue
        if len(letter["AOI"]) == 1:
            for idx in range(len(current_aoi)):
                aoi_clustered.append((len(aoi_clustered), current_aoi[idx], current_left[idx], current_top[idx], current_right[idx], current_bottom[idx], color[idx]))

            current_aoi = []
            color = []
            current_left = []
            current_top = []
            current_right = []
            current_bottom = []
            continue
        # There is no AOI set
        if len(current_aoi) == 0:
            current_aoi = []
            color = []
            current_left = []
            current_top = []
            current_right = []
            current_bottom = []
            for idx in range(1, len(letter["AOI"])):
                current_aoi.append(letter["AOI"][idx])
                current_left.append(letter["BoundingBox"][0])
                current_top.append(letter["BoundingBox"][1])
                current_right.append(letter["BoundingBox"][2])
                current_bottom.append(letter["BoundingBox"][3])
                color.append(letter["color"])
            continue


        for idx in reversed(range(len(current_aoi))):
            if current_aoi[idx] in letter["AOI"]:
                current_left[idx] = min(current_left[idx], letter["BoundingBox"][0])
                current_top[idx] = min(current_top[idx], letter["BoundingBox"][1])
                current_right[idx] = max(current_right[idx], letter["BoundingBox"][2])
                current_bottom[idx] = max(current_bottom[idx], letter["BoundingBox"][3])
                # remove the AOI from the letter
                letter["AOI"].remove(current_aoi[idx])
            else:
                aoi_clustered.append((len(aoi_clustered), current_aoi[idx], current_left[idx], current_top[idx], current_right[idx], current_bottom[idx], color[idx]))
                del current_aoi[idx]
                del current_left[idx]
                del current_top[idx]
                del current_right[idx]
                del current_bottom[idx]
                del color[idx]

        for idx in range(1, len(letter["AOI"])):
            current_aoi.append(letter["AOI"][idx])
            current_left.append(letter["BoundingBox"][0])
            current_top.append(letter["BoundingBox"][1])
            current_right.append(letter["BoundingBox"][2])
            current_bottom.append(letter["BoundingBox"][3])
            color.append(letter["color"])

    for idx in range(len(current_aoi)):
        aoi_clustered.append((len(aoi_clustered), current_aoi[idx], current_left[idx], current_top[idx], current_right[idx], current_bottom[idx], color[idx]))

    for token in aoi_clustered:
        df_aois.loc[len(df_aois)] = [snippet, token[1], token[0],
                                                 (token[2] + width_offset,
                                                  token[3] + height_offset,
                                                  token[4] + width_offset,
                                                  token[5] + height_offset)]

df_aois

  0%|          | 0/32 [00:00<?, ?it/s]

,Algorithm,AOI,AOIIdx,BoundingBox
0,IsPrime,method_identifier,0,"(983, 444, 1060, 460)"
1,IsPrime,method_signature,1,"(741, 444, 1192, 464)"
2,IsPrime,method_argument_declaration,2,"(1060, 444, 1192, 463)"
3,IsPrime,for_head,3,"(785, 469, 1137, 489)"
4,IsPrime,arithmetic_expression,4,"(873, 494, 984, 510)"
...,...,...,...,...
626,Rectangle,method_call_identifier,14,"(1037, 769, 1103, 789)"
627,Rectangle,method_arguments,15,"(1103, 769, 1125, 788)"
628,Rectangle,method_call,16,"(982, 769, 1125, 789)"
629,Rectangle,arithmetic_expression,17,"(817, 769, 1125, 789)"


### Check which Fixation of which Participant is in which AOI

In [9]:
df_aoi_fixation_per_participant = pd.DataFrame([], columns=["Algorithm", "Participant", "FixationNumber",
                                                              "FixationDuration", "AOIIdx", "AOIName"])
participants = df_fixation["Participant"].unique()
for snippet in tqdm(snippets):
    df_aois_per_algo = df_aois[df_aois["Algorithm"] == snippet]

    for participant in tqdm(participants):

        df_fixation_participant = df_fixation[(df_fixation["Algorithm"] == snippet) & (df_fixation["Participant"] == participant)]
        if len(df_fixation_participant) == 0:
            continue
        start_times = eval(df_fixation_participant["Fixation_startT"].values[0])
        end_times = eval(df_fixation_participant["Fixation_endT"].values[0])
        x_coordinates = eval(df_fixation_participant["Fixation_x"].values[0])
        y_coordinates = eval(df_fixation_participant["Fixation_y"].values[0])
        x_range = eval(df_fixation_participant["Fixation_x_range"].values[0])
        y_range = eval(df_fixation_participant["Fixation_y_range"].values[0])
        idx_values = range(len(start_times))
        for (fix_idx, start, end, x, y, x_range, y_range) in zip(idx_values, start_times, end_times, x_coordinates, y_coordinates, x_range, y_range):
            low_x = int(float(x) - math.ceil(float(x_range)))
            low_y = int(float(y) - math.ceil(float(y_range)))
            high_x = int(float(x) + math.ceil(float(x_range)))
            high_y = int(float(y) + math.ceil(float(y_range)))
            possible_coordinates = [(x, y) for x in range(low_x, high_x + 1) for y in range(low_y, high_y + 1)]

            for idx, row in df_aois_per_algo.iterrows():
                aoi_idx = row["AOIIdx"]
                aoi_name = row["AOI"]
                bounding_box = row["BoundingBox"]

                for possible_x, possible_y in possible_coordinates:
                    if bounding_box[0] <= possible_x <= bounding_box[2] and bounding_box[1] <= possible_y <=bounding_box[3]:
                        df_aoi_fixation_per_participant.loc[len(df_aoi_fixation_per_participant)] = [snippet, participant, fix_idx ,end - start, aoi_idx, aoi_name]
                        break

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

...

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]


### Transform the Data to a Fixation/ Refixation split by Participant

In [10]:
df_aois_current = df_aois.copy()
df_aois_current = df_aois_current.drop(["BoundingBox"], axis=1)
for participant in participants:
    df_aois_current.loc[:, f"AOIFixation_P{participant}"] = df_aois_current.apply(lambda row: [], axis=1)
    df_aois_current.loc[:, f"AOIReFixation_P{participant}"] = df_aois_current.apply(lambda row: [], axis=1)


prev_participant = df_aoi_fixation_per_participant["Participant"].iloc[0]
prev_aoi_idx = df_aoi_fixation_per_participant["AOIIdx"].iloc[0]
prev_algorithm = df_aoi_fixation_per_participant["Algorithm"].iloc[0]
fixations = []
re_fixation = False
for idx, row in tqdm(df_aoi_fixation_per_participant.iterrows(), total=len(df_aoi_fixation_per_participant)):
    participant = row["Participant"]
    aoi_idx = row["AOIIdx"]
    algorithm = row["Algorithm"]
    FixationDuration = row["FixationDuration"]

    # fixation switches
    if prev_participant != participant or prev_aoi_idx != aoi_idx:
        index = df_aois_current[(df_aois_current["AOIIdx"] == prev_aoi_idx) & (df_aois_current["Algorithm"] == prev_algorithm)].index[0]
        if re_fixation:
            re_fixations = df_aois_current.loc[index, f"AOIReFixation_P{prev_participant}"]
            re_fixations.extend(fixations.copy())
            df_aois_current.at[index, f"AOIReFixation_P{prev_participant}"] = re_fixations.copy()
        else:
            df_aois_current.at[index, f"AOIFixation_P{prev_participant}"] = fixations.copy()
        fixations = []
        # possible new fixation
        re_fixation = False

    sub_frame = df_aois_current[(df_aois_current["AOIIdx"] == aoi_idx) & (df_aois_current["Algorithm"] == algorithm)]
    if len(sub_frame) == 0:
        raise Exception(f"No AOI found for {aoi_idx} in Algorithm {algorithm}")
    len_of_fixation = len(sub_frame[f"AOIFixation_P{participant}"].iloc[0])
    if re_fixation == False and len_of_fixation > 0 and len(fixations) == 0:
        re_fixation = True

    fixations.append(FixationDuration)
    prev_participant = participant
    prev_aoi_idx = aoi_idx
    prev_algorithm = algorithm

  0%|          | 0/180068 [00:00<?, ?it/s]

### Calculate the AOI Based Eyetracking Metrics

In [17]:
# Melt the Dataframe to be thinner so that we have Algorithm TokenIdx and Participant and Keys per Fixation / Refixation
df_token_melted = pd.melt(df_token, id_vars=["Algorithm", "TokenIdx"], var_name="KindOfFixation", value_name="FixationDurations")

# Classify Participant and the Kind and Number of Fixations / Refixations
df_token_melted["Participant"] = df_token_melted["KindOfFixation"].apply(lambda x: int(x.split("_")[1][1:]))
df_token_melted["KindOfFixation"] = df_token_melted["KindOfFixation"].apply(lambda x: x.split("_")[0])
df_token_melted["KindOfFixation"] = df_token_melted["KindOfFixation"].apply(lambda x: "Fixation" if x == "TokenFixation" else "ReFixation")
df_token_melted["NumberOfFixations"] = df_token_melted["FixationDurations"].apply(lambda x: len(x))

# Get the number of Participants for further calculations
number_of_participants = len(participants)

# Calculate the First Fixation Duration per Participant per Token
df_token_melted["FirstFixationDuration"] = None
df_token_melted.loc[df_token_melted["KindOfFixation"] == "Fixation" , "FirstFixationDuration"] = df_token_melted["FixationDurations"]\
    .apply(lambda x: x[0] if len(x) > 0 else None)

# Calculate the Single Fixation Duration per Participant per Token
df_token_melted["SingleFixationDuration"] = None
df_token_melted.loc[df_token_melted["KindOfFixation"] == "Fixation" , "SingleFixationDuration"] = df_token_melted["FixationDurations"]\
    .apply(lambda x: x[0] if len(x) == 1 else None)

# Calculate the Gaze Duration per Participant per Token
df_token_melted["GazeDuration"] = None
df_token_melted.loc[df_token_melted["KindOfFixation"] == "Fixation" , "GazeDuration"] = df_token_melted["FixationDurations"]\
    .apply(lambda x: sum(x) if len(x) > 0 else None)

# Calculate the Total Time per Participant per Token
df_token_melted_total_time = df_token_melted.groupby(["Participant", "Algorithm", "TokenIdx"])\
    .agg({"FixationDurations": lambda x: sum(x.values.sum())})
df_token_melted_total_time = df_token_melted_total_time.rename(columns={"FixationDurations": "TotalTime"})

# Merge the Dataframes
df_token_melted = pd.merge(df_token_melted, df_token_melted_total_time, on=["Participant", "Algorithm", "TokenIdx"], how="left")

# Cast the Dataframes to the right datatype
df_token_melted["FirstFixationDuration"] = df_token_melted["FirstFixationDuration"].astype(float)
df_token_melted["SingleFixationDuration"] = df_token_melted["SingleFixationDuration"].astype(float)
df_token_melted["GazeDuration"] = df_token_melted["GazeDuration"].astype(float)
df_token_melted["TotalTime"] = df_token_melted["TotalTime"].astype(float).replace(0, np.nan)

# Read in the Skilllevel
df_skill = pd.read_csv(f"../data/24/dataEvaluation/data/filteredData/filtered_data.csv")
df_skill = df_skill[["Participant", "SkillScore"]]
df_skill = df_skill.drop_duplicates()

# Merge the Dataframes to combine metrics with the Skilllevel
df_metrics_skill = pd.merge(df_token_melted, df_skill, on=["Participant"], how="left")

# Helper Methods for the Metrics (2. Define because of nearness of the code)
def get_no_fixations(df):
    df_fixations = df[df["KindOfFixation"] == "Fixation"]
    # remove every entry from df fixations on ["Algorithm", "TokenIdx"] where there is a refixation6
    df_fixations = df_fixations[df_fixations["NumberOfFixations"] == 0]
    return df_fixations


def get_single_fixations(df):
    df_fixations = df[df["KindOfFixation"] == "Fixation"]
    df_refixations = df[df["KindOfFixation"] == "ReFixation"]
    df_refixations = df_refixations[df_refixations["NumberOfFixations"] > 0]
    # remove every entry from df fixations on ["Algorithm", "TokenIdx"] where there is a refixation6
    df_fixations = df_fixations[~df_fixations["TokenIdx"].isin(df_refixations["TokenIdx"].values)]
    # remove every entry from df fixations on where Number Of Fixations is not 1
    df_fixations = df_fixations[df_fixations["NumberOfFixations"] == 1]
    return df_fixations

def get_multiple_fixations(df):
    df_fixations = df[df["KindOfFixation"] == "Fixation"]
    df_refixations = df[df["KindOfFixation"] == "ReFixation"]
    df_refixations = df_refixations[df_refixations["NumberOfFixations"] > 0]
    # remove every entry from df fixations on ["Algorithm", "TokenIdx"] where there is a refixation6
    df_fixations = df_fixations[(df_fixations["TokenIdx"].isin(df_refixations["TokenIdx"].values)) & (df_fixations["NumberOfFixations"] >= 1)]
    return df_fixations

def get_fixations(df):
    df_fixations = df[df["KindOfFixation"] == "Fixation"]
    # remove every entry from df fixations on where no Fixation is found
    df_fixations = df_fixations[df_fixations["NumberOfFixations"] >= 1]
    return df_fixations



# dataframe for number of fixations per participant
number_of_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])["NumberOfFixations"].count()
number_of_fixation_per_algorithm = number_of_fixation_per_algorithm.reset_index()

# dataframe for number of tokens per algorithm
number_of_tokens_per_algorithm = df_metrics_skill.groupby(["Algorithm"])["TokenIdx"].max()
number_of_tokens_per_algorithm = number_of_tokens_per_algorithm.reset_index()

# dataframe for number of tokens with no fixation per algorithm per participant
df_no_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])\
    .apply(get_no_fixations)\
    .drop(["Algorithm"], axis=1)

# dataframe for number of tokens with only one fixation per algorithm per participant
df_single_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])\
    .apply(get_single_fixations)\
    .drop(["Algorithm"], axis=1)

# dataframe for number of tokens with more than one fixation per algorithm per participant
df_multiple_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])\
    .apply(get_multiple_fixations)\
    .drop(["Algorithm"], axis=1)

# dataframe for number of tokens with more or equal than one fixation per algorithm per participant
df_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])\
    .apply(get_fixations)\
    .drop(["Algorithm"], axis=1)

# Reformat the dataframes
no_fixation_per_algorithm = df_no_fixation_per_algorithm[["TokenIdx"]]
no_fixation_per_algorithm = no_fixation_per_algorithm.reset_index().drop(["level_2"], axis=1)

single_fixation_per_algorithm = df_single_fixation_per_algorithm[["TokenIdx"]]
single_fixation_per_algorithm = single_fixation_per_algorithm.reset_index().drop(["level_2"], axis=1)

multiple_fixation_per_algorithm = df_multiple_fixation_per_algorithm[["TokenIdx"]]
multiple_fixation_per_algorithm = multiple_fixation_per_algorithm.reset_index().drop(["level_2"], axis=1)

fixations_per_algorithm = df_fixation_per_algorithm[["TokenIdx"]]
fixations_per_algorithm = fixations_per_algorithm.reset_index().drop(["level_2"], axis=1)

# Helper Method for Probability Metrics
def group_len_divided_by_number(current_df, counting_df):
    algorithm = current_df["Algorithm"].iloc[0]
    number_of_tokens = counting_df[counting_df["Algorithm"] == algorithm]["TokenIdx"].iloc[0]
    value = len(current_df) / number_of_tokens
    return len(current_df) / (number_of_tokens + 1)

# Calculate the Metrics per Participant
# Probability of no fixation
df_no_fixation_probability = no_fixation_per_algorithm.groupby(["Participant", "Algorithm"]).apply(lambda df: group_len_divided_by_number(df, number_of_tokens_per_algorithm))
df_no_fixation_probability = df_no_fixation_probability.reset_index()

# Probability of single fixation
df_single_fixation_probability = single_fixation_per_algorithm.groupby(["Participant", "Algorithm"]).apply(lambda df: group_len_divided_by_number(df, number_of_tokens_per_algorithm))
df_single_fixation_probability = df_single_fixation_probability.reset_index()

# Probability of multiple fixation
df_multiple_fixation_probability = multiple_fixation_per_algorithm.groupby(["Participant", "Algorithm"]).apply(lambda df: group_len_divided_by_number(df, number_of_tokens_per_algorithm))
df_multiple_fixation_probability = df_multiple_fixation_probability.reset_index()

# Probability of fixation
df_fixation_probability = fixations_per_algorithm.groupby(["Participant", "Algorithm"]).apply(lambda df: group_len_divided_by_number(df, number_of_tokens_per_algorithm))
df_fixation_probability = df_fixation_probability.reset_index()

# Calculate the means for the metrics per algorithm
df_no_fixation_probability = df_no_fixation_probability.groupby(["Participant"]).mean(numeric_only=True)
df_single_fixation_probability = df_single_fixation_probability.groupby(["Participant"]).mean(numeric_only=True)
df_multiple_fixation_probability = df_multiple_fixation_probability.groupby(["Participant"]).mean(numeric_only=True)
df_fixation_probability = df_fixation_probability.groupby(["Participant"]).mean(numeric_only=True)


if 41 not in df_multiple_fixation_probability.index:
    df_multiple_fixation_probability.loc[41] = 0

# Raw Durations Metrics
# Duration of first fixation
df_first_fixation = df_metrics_skill[~df_metrics_skill["FirstFixationDuration"].isnull()]
df_first_fixation = df_first_fixation.groupby(["Participant"])["FirstFixationDuration"].mean()

# Duration of single fixation
df_single_fixation = df_metrics_skill[~df_metrics_skill["SingleFixationDuration"].isnull()]
df_single_fixation = df_single_fixation.groupby(["Participant"])["SingleFixationDuration"].mean()

# Duration of gaze duration
df_gaze_duration = df_metrics_skill[~df_metrics_skill["GazeDuration"].isnull()]
df_gaze_duration = df_gaze_duration.groupby(["Participant"])["GazeDuration"].mean()

# Total Time
df_total_time = df_metrics_skill[~df_metrics_skill["TotalTime"].isnull()]
df_total_time = df_total_time.groupby(["Participant"])["TotalTime"].mean()

# Put every metric dataframe together into one
df_combined = pd.DataFrame({"FirstFixationDuration": df_first_fixation.values,
                            "SingleFixationDuration": df_single_fixation.values,
                            "GazeDuration": df_gaze_duration.values,
                            "TotalTime": df_total_time.values,
                            "TokenNoFixationProbability": df_no_fixation_probability.values.reshape(37, ),
                            "TokenSingleFixationProbability": df_single_fixation_probability.values.reshape(37, ),
                            "TokenMultipleFixationProbability": df_multiple_fixation_probability.values.reshape(37, ),
                            "TokenFixationProbability": df_fixation_probability.values.reshape(37, ),
                            "Skill": df_metrics_skill.groupby(["Participant"])["SkillScore"].mean().values})
# get spearman correlation for metrics and skill level
df_combined.corrwith(df_metrics_skill["SkillScore"])

[ 1  2  3  4  5  6  7 10 11 12 13 14 18 22 24 25 28 35 36 37 38 42 49 50
 55 58 59 60 61 62 63 66 67 68 70 71]
[ 1  2  3  4  5  6  7 10 11 12 13 14 18 22 24 25 28 35 36 37 38 41 42 49
 50 55 58 59 60 61 62 63 66 67 68 70 71]


FirstFixationDuration               1.531731e-16
SingleFixationDuration              2.971250e-16
GazeDuration                        8.830300e-16
TotalTime                           3.968537e-16
TokenNoFixationProbability          2.821054e-15
TokenSingleFixationProbability      5.870057e-17
TokenMultipleFixationProbability   -1.715243e-17
TokenFixationProbability           -3.435045e-16
Skill                              -5.638273e-16
dtype: float64

# Get the LOCs

In [19]:
# Get Bounding Boxes for Lines Of Code
snippets = df_fixation["Algorithm"].unique()
df_lines = pd.DataFrame(columns=["Algorithm", "Line", "BoundingBox"])
for snippet in tqdm(snippets):
    aoi_token_generator = f"../data/24/CodeSnippets/Generators_Labeled/Generators/{snippet}.json"
    image, aoi_list = gsl.create_image(aoi_token_generator, font_path="/../data/24/CodeSnippets/fonts/ttf/")
    height, width = image.size
    width_offset = int(1920 * 0.5) - int(height / 2)
    height_offset = int(1080 * 0.5) - int(width / 2)
    aoi_clustered = []
    current_left = None
    current_top = None
    current_right = None
    current_bottom = None
    current_line = 0
    for letter in aoi_list:
        if letter["letter"] == '\n':
            if current_left is not None:
                aoi_clustered.append((current_line, current_left, current_top, current_right, current_bottom))
            current_left = None
            current_top = None
            current_right = None
            current_bottom = None
            current_line += 1
            continue
        if current_left is None:
            current_left = letter["BoundingBox"][0]
            current_top = letter["BoundingBox"][1]
            current_right = letter["BoundingBox"][2]
            current_bottom = letter["BoundingBox"][3]
        else:
            current_left = min(current_left, letter["BoundingBox"][0])
            current_top = min(current_top, letter["BoundingBox"][1])
            current_right = max(current_right, letter["BoundingBox"][2])
            current_bottom = max(current_bottom, letter["BoundingBox"][3])

    for token in aoi_clustered:
        df_lines.loc[len(df_lines)] = [snippet, token[0],
                                       (token[1] + width_offset,
                                        token[2] + height_offset,
                                        token[3] + width_offset,
                                        token[4] + height_offset)]
df_lines

  0%|          | 0/32 [00:00<?, ?it/s]

,Algorithm,Line,BoundingBox
0,IsPrime,0,"(686, 443, 1214, 464)"
1,IsPrime,1,"(686, 468, 1159, 489)"
2,IsPrime,2,"(686, 493, 1072, 513)"
3,IsPrime,3,"(686, 521, 1005, 539)"
4,IsPrime,4,"(686, 543, 829, 562)"
...,...,...,...
432,Rectangle,19,"(652, 693, 707, 712)"
433,Rectangle,21,"(652, 743, 938, 764)"
434,Rectangle,22,"(652, 769, 1136, 789)"
435,Rectangle,23,"(652, 793, 707, 812)"


In [20]:
df_line_fixation_per_participant = pd.DataFrame([], columns=["Algorithm", "Participant", "FixationNumber", "FixationStart", "FixationEnd", "LineNumber"])
participants = df_fixation["Participant"].unique()
for snippet in tqdm(snippets):
    df_token_per_algo = df_lines[df_lines["Algorithm"] == snippet]

    for participant in participants:
        df_fixation_participant = df_fixation[(df_fixation["Algorithm"] == snippet) & (df_fixation["Participant"] == participant)]
        if len(df_fixation_participant) == 0:
            continue
        start_times = eval(df_fixation_participant["Fixation_startT"].values[0])
        end_times = eval(df_fixation_participant["Fixation_endT"].values[0])
        y_coordinates = eval(df_fixation_participant["Fixation_y"].values[0])
        y_range = eval(df_fixation_participant["Fixation_y_range"].values[0])
        idx_values = range(len(start_times))
        for (fix_idx, start, end, y, y_range) in zip(idx_values, start_times, end_times, y_coordinates, y_range):
            low_y = int(float(y) - math.ceil(float(y_range)))
            high_y = int(float(y) + math.ceil(float(y_range)))
            possible_coordinates = [y for y in range(low_y, high_y + 1)]

            found = False
            for idx, row in df_token_per_algo.iterrows():
                line_number = row["Line"]
                bounding_box = row["BoundingBox"]

                for possible_y in possible_coordinates:
                    if bounding_box[1] <= possible_y <= bounding_box[3]:
                        df_line_fixation_per_participant.loc[len(df_line_fixation_per_participant)] = [snippet, participant, fix_idx, start, end, line_number]
                        found = True
                        break
                if found:
                    break

df_line_fixation_per_participant

  0%|          | 0/32 [00:00<?, ?it/s]

,Algorithm,Participant,FixationNumber,FixationStart,FixationEnd,LineNumber
0,IsPrime,1,0,0,240,0
1,IsPrime,1,1,252,740,0
2,IsPrime,1,2,768,996,0
3,IsPrime,1,3,1236,1348,1
4,IsPrime,1,4,1360,1532,1
...,...,...,...,...,...,...
65065,Rectangle,71,22,7964,8124,13
65066,Rectangle,71,24,8540,8764,21
65067,Rectangle,71,25,8832,8944,8
65068,Rectangle,71,27,9460,9596,8


In [22]:
# Calculate the LOCs
snippets = df_fixation["Algorithm"].unique()
df_snippet_length = pd.DataFrame(columns=["Algorithm", "LOC"])
for snippet in tqdm(snippets):
    aoi_token_generator = f"../data/24/CodeSnippets/Generators_Labeled/Generators/{snippet}.json"
    with open(aoi_token_generator) as f:
        aoi_list = json.load(f)
        data = aoi_list["source-code"]
        LOC = len(data)
        df_snippet_length.loc[len(df_snippet_length)] = [snippet, LOC]
df_snippet_length

  0%|          | 0/32 [00:00<?, ?it/s]

,Algorithm,LOC
0,IsPrime,8
1,SiebDesEratosthenes,20
2,IsAnagram,28
3,RemoveDoubleChar,19
4,BinToDecimal,9
5,PermuteString,23
6,Power,7
7,BinarySearch,15
8,ContainsSubstring,21
9,ReverseArray,7


In [23]:
# read in Behavioural and Skills data
df_behavioral = pd.read_csv('../data/24/dataEvaluation/data/filteredData/fixation_stats.csv', sep=";")
df_behavioral = df_behavioral[df_behavioral["IsOutlier"] == False]
df_behavioral = df_behavioral[["Participant", "Algorithm", "Duration", "SkillScore"]]
df_behavioral

,Participant,Algorithm,Duration,SkillScore
0,1,IsPrime,12.390280,0.331385
1,1,SiebDesEratosthenes,152.571914,0.331385
2,1,IsAnagram,109.615724,0.331385
3,1,RemoveDoubleChar,53.456276,0.331385
4,1,BinToDecimal,49.922091,0.331385
...,...,...,...,...
1066,71,GreatestCommonDivisor,30.757360,0.435651
1067,71,DumpSorting,113.368945,0.435651
1068,71,BinomialCoefficient,50.637861,0.435651
1069,71,IsAnagram,110.995754,0.435651


In [24]:
# merge dataframes together to access every possible combination of participant, algorithm and duration,skillscore
df_combined = pd.merge(df_line_fixation_per_participant, df_snippet_length, on=["Algorithm"])
df_combined = pd.merge(df_combined, df_behavioral, on=["Participant", "Algorithm"])

# transform the data to seconds
df_combined["FixationStart"] = df_combined["FixationStart"] / 1000.0
df_combined["FixationEnd"] = df_combined["FixationEnd"] / 1000.0

# Helper function to calculate the coverage of LOC after 'percentage' percent from the whole duration
def loc_coverage_after_time_percentage(df, percentage):
    end_duration = df["Duration"].iloc[0]
    loc = df["LOC"].iloc[0]
    max_duration = end_duration * percentage
    df_filtered = df[df["FixationEnd"] <= max_duration]
    unique_loc = df_filtered["LineNumber"].nunique()
    return unique_loc / loc

# calculate LOC coverage after a certain percentage of the duration
df_10 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.1))
df_20 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.2))
df_30 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.3))
df_40 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.4))
df_50 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.5))
df_60 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.6))
df_70 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.7))
df_80 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.8))
df_90 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 0.9))
df_100 = df_combined.groupby(["Algorithm", "Participant"]).apply(lambda df : loc_coverage_after_time_percentage(df, 1.))

df_10 = df_10.reset_index()
df_20 = df_20.reset_index()
df_30 = df_30.reset_index()
df_40 = df_40.reset_index()
df_50 = df_50.reset_index()
df_60 = df_60.reset_index()
df_70 = df_70.reset_index()
df_80 = df_80.reset_index()
df_90 = df_90.reset_index()
df_100 = df_100.reset_index()


df_10 = df_10.groupby(["Participant"]).mean(numeric_only=True).values.reshape(37, )
df_20 = df_20.groupby(["Participant"]).mean(numeric_only=True).values.reshape(37, )
df_30 = df_30.groupby(["Participant"]).mean(numeric_only=True).values.reshape(37, )
df_40 = df_40.groupby(["Participant"]).mean(numeric_only=True).values.reshape(37, )
df_50 = df_50.groupby(["Participant"]).mean(numeric_only=True).values.reshape(37, )
df_60 = df_60.groupby(["Participant"]).mean(numeric_only=True).values.reshape(37, )
df_70 = df_70.groupby(["Participant"]).mean(numeric_only=True).values.reshape(37, )
df_80 = df_80.groupby(["Participant"]).mean(numeric_only=True).values.reshape(37, )
df_90 = df_90.groupby(["Participant"]).mean(numeric_only=True).values.reshape(37, )
df_100 = df_100.groupby(["Participant"]).mean(numeric_only=True).values.reshape(37, )

# extract the raw skillscore per participant
df_skill = df_behavioral[["Participant", "SkillScore"]]
df_skill = df_skill.drop_duplicates()

# create a dataframe with all the LOC coverage and Participant
df_code_coverage = pd.DataFrame({"Participant": participants,
                                 "10%" : df_10,
                                 "20%" : df_20,
                                 "30%" : df_30,
                                 "40%" : df_40,
                                 "50%" : df_50,
                                 "60%" : df_60,
                                 "70%" : df_70,
                                 "80%" : df_80,
                                 "90%" : df_90,
                                 "100%" : df_100})

df_code_coverage.set_index("Participant", inplace=True, drop=True)

# Merge LOC coverage with skillscore with correlations
df_code_coverage = pd.merge(df_code_coverage, df_skill, on=["Participant"])
df_code_coverage.set_index("Participant", inplace=True, drop=True)
df_code_coverage

,10%,20%,30%,40%,50%,60%,70%,80%,90%,100%,SkillScore
Participant,,,,,,,,,,,
1,0.208355,0.333523,0.411686,0.484565,0.540732,0.595668,0.620258,0.642353,0.678408,0.724189,0.331385
2,0.173380,0.242968,0.320055,0.394289,0.421425,0.463251,0.521250,0.554454,0.601129,0.634964,0.379187
3,0.185040,0.245763,0.307408,0.360887,0.394680,0.424992,0.443479,0.479749,0.510981,0.541787,0.311264
4,0.122569,0.277799,0.363282,0.446762,0.495006,0.528270,0.563911,0.596592,0.618127,0.653717,0.424727
5,0.125606,0.198424,0.269902,0.346634,0.389048,0.451111,0.478704,0.503806,0.543484,0.590112,0.313031
6,0.138565,0.282328,0.374619,0.459819,0.536354,0.566045,0.607339,0.636138,0.655154,0.690935,0.315932
7,0.091716,0.120997,0.149592,0.164705,0.195754,0.213534,0.247203,0.264929,0.282104,0.295801,0.420873
10,0.102075,0.169259,0.235940,0.329361,0.401875,0.456438,0.505837,0.565355,0.603250,0.621506,0.350392
11,0.141498,0.288044,0.454891,0.548636,0.589183,0.638667,0.702683,0.733899,0.768822,0.798800,0.178206


In [25]:
# describe the LOC coverage
df_code_coverage.describe()

,10%,20%,30%,40%,50%,60%,70%,80%,90%,100%,SkillScore
count,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000
mean,0.124588,0.211593,0.277882,0.337951,0.389867,0.428402,0.467936,0.500209,0.531910,0.570191,0.325230
std,0.075141,0.090387,0.106836,0.121247,0.130022,0.136918,0.145383,0.152005,0.158980,0.165885,0.102510
min,0.004464,0.042411,0.042411,0.048990,0.059252,0.059252,0.063717,0.063717,0.076705,0.076705,0.142520
25%,0.063186,0.137744,0.212560,0.263874,0.339800,0.375956,0.441298,0.479749,0.510981,0.553602,0.259896
50%,0.120929,0.239413,0.287475,0.346634,0.393192,0.447146,0.482840,0.528462,0.569690,0.605892,0.315932
75%,0.172414,0.282328,0.355388,0.427825,0.480979,0.519832,0.563911,0.587145,0.620209,0.668242,0.379187
max,0.352598,0.445310,0.541943,0.591981,0.653982,0.664387,0.704810,0.733899,0.768822,0.798800,0.656726


In [37]:
# spearman correlation for LOC coverage and skillscore
LOC_corr = df_code_coverage.corrwith(df_code_coverage["SkillScore"], method = "spearman")
LOC_corr

10%          -0.004979
20%          -0.064486
30%          -0.073732
40%          -0.071361
50%          -0.088431
60%          -0.094832
70%          -0.125415
80%          -0.155050
90%          -0.158369
100%         -0.228307
SkillScore    1.000000
dtype: float64

In [36]:
# Save the Dataframe to a CSV
LOC_corr.to_csv("results/ogama_code_coverage.csv", index=True)

In [28]:
from matplotlib.patches import Circle
import matplotlib.pyplot as plt
import os


# check if folder exists and create it if not
if not os.path.exists("./data/RQ1/plots_ogama/scanpaths"):
    os.makedirs("./data/RQ1/plots_ogama/scanpaths")

if not os.path.exists("./data/RQ1/plots_ogama/vertical_scanpaths"):
    os.makedirs("./data/RQ1/plots_ogama/vertical_scanpaths")

def clamp(low, value, high):
    return max(low, min(value, high))

snippets = df_fixation["Algorithm"].unique()
# iterate over all algorithms
for snippet in tqdm(snippets):
    aoi_token_generator = f"../data/24/CodeSnippets/Generators_Labeled/Generators/{snippet}_ast.json"
    image, aoi_list = gsl.create_image(aoi_token_generator, font_path="/../data/24/CodeSnippets/fonts/ttf/")
    width, height = image.size
    x_low = int(1920 * 0.5) - int(width / 2)
    y_low = int(1080 * 0.5) - int(height / 2)
    x_high = int(1920 * 0.5) + int(width / 2)
    y_high = int(1080 * 0.5) + int(height / 2)
    df_algo = df_fixation[df_fixation["Algorithm"] == snippet]
    df_algo = df_algo.sort_values(by=["SkillScore"])
    participants = df_algo["Participant"].unique()
    # iterate over all participants
    for participant in participants:
        if len(df_algo[df_algo["Participant"] == participant]) != 1:
            display(df_algo[df_algo["Participant"] == participant])
            raise ValueError("Participant has more than not exactly one entry")
        df_algo_part = df_algo[df_algo["Participant"] == participant].reset_index().iloc[0]
        skillscore = df_algo_part["SkillScore"]
        current_image = image.copy()
        fixation_start_array = np.array(eval(df_algo_part["Fixation_startT"]))
        fixation_end_array = np.array(eval(df_algo_part["Fixation_endT"]))
        fixation_x_coordinates = np.array(eval(df_algo_part["Fixation_x"]))
        fixation_y_coordinates = np.array(eval(df_algo_part["Fixation_y"]))
        fixations = np.stack((fixation_start_array, fixation_end_array, fixation_x_coordinates, fixation_y_coordinates), axis=1)

        cm = plt.cm.get_cmap('inferno')
        cm = plt.cm.ScalarMappable(cmap=cm, norm=plt.Normalize(vmin=0, vmax=len(fixations)))
        patches = []

        # total scanpath
        for idx, (start, end, x, y) in enumerate(fixations):
            if x_low <= x <= x_high and y_low <= y <= y_high:
                x = int(x)
                y = int(y)
                x = x - x_low
                y = y - y_low
                color = cm.to_rgba(idx)
                patches.append(Circle((x, y), radius=10, color=color, alpha=0.5))

        fig, ax = plt.subplots(1)
        ax.imshow(current_image)
        for p in patches:
            ax.add_patch(p)

        for idx in range(len(fixations) - 1):
            start_x = fixations[idx, 2] - x_low
            start_y = fixations[idx, 3] - y_low
            end_x = fixations[idx+1, 2] - x_low
            end_y = fixations[idx+1, 3] - y_low
            if 0 <= start_x <= width and 0 <= start_y <= height and 0 <= end_x <= width and 0 <= end_y <= height:
                color = cm.to_rgba(idx)
                ax.plot([start_x, end_x], [start_y, end_y], color=color, alpha=0.3)

        plt.xlim(0, width)
        plt.ylim(height, 0)
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(f"./data/RQ1/plots_ogama/scanpaths/scanpath_{snippet}_{skillscore}_{participant}.pdf")
        plt.close()

        # vertical scanpath
        vertical_points_and_time = []
        for idx, (start, end, x, y) in enumerate(fixations):
            if x_low <= x <= x_high and y_low <= y <= y_high:
                y = int(y) - y_low
                duration = (end - start) / 1000
                vertical_points_and_time.append((y, duration))

        pixel_per_second = 3
        current_x = 0
        if len(vertical_points_and_time) == 0:
            continue
        line_points = [(current_x, vertical_points_and_time[0][0])]
        current_x += vertical_points_and_time[0][1] * pixel_per_second
        line_points.append((current_x, vertical_points_and_time[0][0]))
        for idx, (y, duration) in enumerate(vertical_points_and_time[1:]):
            line_points.append((current_x, y))
            current_x += duration * pixel_per_second
            line_points.append((current_x, y))

        max_x = max([x for x, y in line_points])
        line_points = [(x - max_x, y) for x, y in line_points]
        cm = plt.cm.get_cmap('inferno')
        cm = plt.cm.ScalarMappable(cmap=cm, norm=plt.Normalize(vmin=0, vmax=len(line_points)))
        fig, ax = plt.subplots(1)
        ax.imshow(current_image)
        for idx in range(len(line_points) - 1):
            start_x = line_points[idx][0]
            start_y = line_points[idx][1]
            end_x = line_points[idx+1][0]
            end_y = line_points[idx+1][1]
            color = cm.to_rgba(idx)
            ax.plot([start_x, end_x], [start_y, end_y], color=color)
        plt.xlim(-max_x, width)
        plt.ylim(height, 0)
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(f"./data/RQ1/plots_ogama/vertical_scanpaths/scanpath_vertical_{snippet}_{skillscore}_{participant}.pdf", dpi=200)
        plt.close()

  0%|          | 0/32 [00:00<?, ?it/s]

C:\Users\Timon\AppData\Local\Temp\ipykernel_50860\1359144161.py:43: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykernel_50860\1359144161.py:100: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykernel_50860\1359144161.py:43: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykerne

AOI not closed - None


C:\Users\Timon\AppData\Local\Temp\ipykernel_50860\1359144161.py:43: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykernel_50860\1359144161.py:100: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykernel_50860\1359144161.py:43: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykerne

AOI not closed - None


C:\Users\Timon\AppData\Local\Temp\ipykernel_50860\1359144161.py:43: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykernel_50860\1359144161.py:100: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykernel_50860\1359144161.py:43: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykerne

AOI not closed - None


C:\Users\Timon\AppData\Local\Temp\ipykernel_50860\1359144161.py:43: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykernel_50860\1359144161.py:100: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykernel_50860\1359144161.py:43: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cm = plt.cm.get_cmap('inferno')
C:\Users\Timon\AppData\Local\Temp\ipykerne